# Reprocess -raw.fif to -epo.fif using existing per-subject information

In [1]:
import sys
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import scipy.io as scio
from IPython import display

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def get_time_window():
    bmax=0.
    t_win = float(input("\nChoose a peri-stimulus time window."+
    "\nEx: '0 (default)' = [-0.2,0.8], '2' = [-1.0,1.0], etc...\n\n>> "))
    if t_win==0.:
        tmin,tmax = -0.2,0.8
    else:
        tmin,tmax = -t_win/2,t_win/2
    print(f"[{tmin},{bmax},{tmax}]")
    sub_save_path=f'{int(t_win)}_sec_time_window/'
    # print(sub_save_path)
    return tmin,bmax,tmax,sub_save_path

In [4]:
def get_complete_subjects(data_path,sub_ids_lst):
    complete_subs=[]
    for fname in os.listdir(data_path):
        if fname.endswith("_preprocessed-epo.fif"):
            complete_subs.append(fname[:3])

    incomplete_subs = set(sub_ids_lst) - set(complete_subs)
    # return list(incomplete_subs)
    print(incomplete_subs)

In [5]:
#### all subject file names in directory for reference
sub_ids_lst = [
 # '006', X
 '007', #32ch
 '008', #32ch
 '009', #32ch
 '010', #32ch
 '013', #32ch
 '014', #32ch
 '015', #32ch
 # '016', X
 # '017', X
 '018',
 # '019', X
 '020',
 '021',
 '022',
 '023',
 '024',
 # '025', X
 # '026', X
 '027',
 # '028', X
 '029',
 '030',
 '031',
 '032',
 '033',
 '034',
 '035',
 '036',
 '037',
 '038',
 '039',
 '040',
 '041',
 '042',
 '043',
 '044',
 '045',
 '046',
 # '047', X
 '048',
 '049',
 '050',
 '051',
 '052',
 '053',
 '054',
 '055',
 '056',
 '057',
 '058',
 '059',
 'C1.',
 'C2.', 
 'C3.',  
 # 'C4.', X
 'C5.', 
 'C6.',
 'C7.',
 'C8.',
 'C9.',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'C15',
 'C16',
 'C17',  
 'C18',
 'C19',
 'C20',
 'C21',
 'C22',
 # 'C23', X
 'C24',
 'C25',
 'C26',
 'C27',
]

In [36]:
fp1_sub_ids=['007', '008', '009', '013', '014', '015',
            '010',
            ]

In [37]:
sub_ids_lst=fp1_sub_ids

In [38]:
data_path = "../../Chronic Low Back Pain EEG/Data/Processed Data/"

In [39]:
get_complete_subjects(data_path,sub_ids_lst)

set()


In [40]:
# Choose peri-stimulus time window
tmin,bmax,tmax,sub_save_path = get_time_window()


Choose a peri-stimulus time window.
Ex: '0 (default)' = [-0.2,0.8], '2' = [-1.0,1.0], etc...

>>  5


[-2.5,0.0,2.5]


In [42]:
save_path = "../../Chronic Low Back Pain EEG/Data/Processed Data/" + sub_save_path

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [43]:
for fname in os.listdir(data_path):
    sub_id='' # reset
    save_fname = fname[:3] + "_preprocessed-epo.fif"
    print(save_fname)
    if ( fname[:3] in sub_ids_lst and # if not already processed
        fname.endswith('raw.fif') and 
        save_fname not in os.listdir(save_path) ):
        sub_id = fname[:3]
        print(f'{sub_id}\nreading preprocessed-raw file...')
        raw = mne.io.read_raw_fif(data_path+fname, preload=True) # load raw
    else:
        continue
    display.clear_output(wait=True)

    # Load trial information
    print(f'Reading stim_labels & epo_times for subject {sub_id}...')
    
    stim_labels = scio.loadmat(data_path+sub_id+'_stim_labels.mat')
    stim_labels = stim_labels['stim_labels'].tolist()
    if type(stim_labels[0]) == list and len(stim_labels[0])==1:
        stim_labels_new = [el[0] for el in stim_labels]
        stim_labels=stim_labels_new
    elif type(stim_labels[0]) == list and len(stim_labels[0])>1:
        stim_labels=stim_labels[0]
    print(f'\n*len(stim_labels) = {len(stim_labels)}*')

    epo_times = scio.loadmat(data_path+sub_id+'_epo_times.mat')
    epo_times_raw = epo_times['epo_times'] # leave as array
    epo_times = [el[0] for el in epo_times_raw]
    print(f'*len(epo_times) = {len(epo_times)}*\n')

    display.clear_output(wait=True)

    # create events array with dummy labels
    events=np.array([[epo_times[i],0,3] for i in range(len(epo_times))])
        
    # create epochs object
    epochs = mne.Epochs(raw, events, event_id=None, 
                        tmin=tmin, tmax=tmax, 
                        proj=False, preload=True)
    
    # replace missing event information
    events_new=np.array([[epo_times[i],0,stim_labels[i]] for i in range(len(epo_times))])
    if (sub_id=='020' or sub_id=='018' or sub_id=='C1.' or sub_id=='014' or
        sub_id=='C2.' or sub_id=='C3.' or sub_id=='C4.' or sub_id=='C5.'):
         event_id = {
                  'Hand HS': 3, # hand YES pain
                  'Hand NS': 5, # hand NO pain
                  'Back HS': 6, # back YES pain
                  'Back NS': 8, # back NO pain
                }
    else: 
        event_id = {
                  'Hand HS': 3, # hand YES pain
                  'Hand LS': 4, # hand MED pain
                  'Hand NS': 5, # hand NO pain
                  'Back HS': 6, # back YES pain
                  'Back LS': 7, # back MED pain
                  'Back NS': 8, # back NO pain
                }
    epochs.events = events_new
    epochs.event_id = event_id
    # break
    # save
    epochs.save(save_path + save_fname,
                     verbose=True,
                     overwrite=False)

Not setting metadata
67 matching events found
Setting baseline interval to [-2.5, 0.0] s
Applying baseline correction (mode: mean)
Using data from preloaded Raw for 67 events and 2001 original time points ...
0 bad epochs dropped
052_preprocessed-epo.fif
C26_preprocessed-epo.fif
C11_preprocessed-epo.fif
C27_preprocessed-epo.fif
058_preprocessed-epo.fif
045_preprocessed-epo.fif
056_preprocessed-epo.fif
046_preprocessed-epo.fif
C15_preprocessed-epo.fif
C26_preprocessed-epo.fif
015_preprocessed-epo.fif
C21_preprocessed-epo.fif
027_preprocessed-epo.fif
C12_preprocessed-epo.fif
013_preprocessed-epo.fif
C6._preprocessed-epo.fif
041_preprocessed-epo.fif
015_preprocessed-epo.fif
018_preprocessed-epo.fif
C1._preprocessed-epo.fif
C17_preprocessed-epo.fif
009_preprocessed-epo.fif
C25_preprocessed-epo.fif
048_preprocessed-epo.fif
052_preprocessed-epo.fif
040_preprocessed-epo.fif
023_preprocessed-epo.fif
C24_preprocessed-epo.fif
C10_preprocessed-epo.fif
032_preprocessed-epo.fif
014_preprocessed-epo

In [17]:
len(epo_times)

65

In [19]:
len(stim_labels)

64